Ex 4.9 Lisa Coombs Intro to Data Visualizations with Python

This script contains the following for PART 1:
1. Import Libraries
2. Create Path
3. Import Customer Data
4. Data Exploration
5. Wrangle Data (Ex 4.4)
6. Data Quality & Consistency Checks (Ex 4.5)
7. Combine customer data with ords_prods data (Ex 4.6)
8. Export new file (ords_prods_cust) as .pkl file to "Prepared Data"

1. ***Import Libraries***

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

2. ***Create Path***

In [5]:
path=r'C:\Users\lisac\10-10-2024 Instacart Basket Analysis'

3. ***Import Customer Data***

In [7]:
cust_data=pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

4. ***Data Exploration***

In [9]:
#Exploring different aspects of customer data to get an idea of what is in the dataframe
cust_data.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Observation: This data looks like details for each customer in the dataframe.  Interesting that Instacart included dependents for each of their customers.

In [11]:
cust_data.tail()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799
206208,80148,Cynthia,Noble,Female,New York,55,4/1/2020,1,married,57095


Observation:  There are 206,208 rows of customers.

In [13]:
#Total number of rows and columns
cust_data.shape

(206209, 10)

In [14]:
#Getting a look at the basic information about the dataframe
cust_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


Observation:  This is s a Pandas dataframe.  There are 206209 entries (rows) with idexing starting from 0 to 206209.  Ten (10) columns and 
the datatypes are int64 and object (objects in Pandas is a string).  The dataframe takes up approx 16 MB of memory on my laptop.  It's interesting that the dtype for what I would think is a string is listed as an object.

There looks to be missing values in the First Name column.

In [16]:
#Checking the .value_counts() of the First Name column
cust_data['First Name'].apply(type).value_counts()

First Name
<class 'str'>      194950
<class 'float'>     11259
Name: count, dtype: int64

Observation:  There is a mixed datatype in the First Name column.  Will address this in the Data Quality and Consistency section of this script

In [18]:
#Descriptive statistics
cust_data.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


Observations: This summary shows statistics for 4 numerical colums:  user_id, Age, n_dependents, and income.

Column analysis:
user_id:  there are 206209 entries, with no missing values.
        Range:  the ID's span from 1 to 206209
        
        Insight:  This field looks like an identifier field, with no other significant insight

Age:  the average age is approx 50 years old

        Range:  18 to 81 years old
        
        Quartiles:  25th is 33 years old
                    50th is 49 years old
                    75th is 66 years old
                    
        Insight:  It looks like most of the customers are middle-aged, but there is a significant representation of younger and older customers

n_dependents:  the average number of dependents per customer is approx 1.5. Since you really can't have .5 of a human, these numbers account for variations in the data, ie having 1 or 3 dependents (avg is 1.5).

        Std Deviation:  is approx 1.12 which indicates that customers are likely to have 1 or 2 dependents.  There are fewer customers that have 0 or 3 dependents.

income:  the average income is approx 95,000

        Range: approx 26,000 to 594,000
        
        Std Deviation:  approx 42,000, which indicates a widespread range of incomes
        
        Quartiles:  25th is approx 60,000
                    50th is approx 94,000
                    75th is approx 124,000
                    
        Insights:  it looks like the income distribution leans towards higher earners, however, there does seem to be income disparity.

5. ***Wrangle Data***

In [21]:
#Review headers for renaming
cust_data.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [22]:
#Renaming columns
cust_data = cust_data.rename(columns={ 
    'Surnam' : 'Last Name', 
    'STATE' : 'State', 
    'date_joined' : 'Date Joined', 
    'n_dependants' : 'Number of Dependents', 
    'fam_status' : 'Family Status', 
    'income' : 'Income',
    'user_id' : 'Customer_ID'})

In [23]:
cust_data.head()

,Customer_ID,First Name,Last Name,Gender,State,Age,Date Joined,Number of Dependents,Family Status,Income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Observation:  All header names are uniform and plainly descriptive.  I made sure to change the User_ID to Customer_ID as this column will act as a "key characteristic" when I merge the cust_data dataframe with the ords_prods_merge dataframe.

6. ***Data Quality & Consistency Checks***

In [26]:
#Taking a second look at missing values in First Name column
cust_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Customer_ID           206209 non-null  int64 
 1   First Name            194950 non-null  object
 2   Last Name             206209 non-null  object
 3   Gender                206209 non-null  object
 4   State                 206209 non-null  object
 5   Age                   206209 non-null  int64 
 6   Date Joined           206209 non-null  object
 7   Number of Dependents  206209 non-null  int64 
 8   Family Status         206209 non-null  object
 9   Income                206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


Observation:  looking at the dtype "object" for columns like First Name and Last Name.  Shouldn't these be strings?  Googled what this means 
and got:  

In pandas, the object dtype is a general-purpose type that can hold any Python objects. When strings are stored in a DataFrame, they 
are represented as object. Therefore, the columns with object data type likely contain strings.

Additionally, I think there may be some missing values in the First Name column.

In [28]:
#Adressing the mixed datatype in the First Name column using the .value_counts() function
cust_data['First Name'].apply(type).value_counts()

First Name
<class 'str'>      194950
<class 'float'>     11259
Name: count, dtype: int64

In [29]:
#Counting missing values
cust_data.isnull().sum()

Customer_ID                 0
First Name              11259
Last Name                   0
Gender                      0
State                       0
Age                         0
Date Joined                 0
Number of Dependents        0
Family Status               0
Income                      0
dtype: int64

Observation:  it looks like there are 11,259 instances where a customer's first name is missing.  This probably won't hinder my analysis.  I don't think there is a need to impute Names (strings) for this column, however, the mixed datatype does need to be addressed by making it uniform.  I will make it a string as this is a column for names.

In [31]:
#Changing the datatype in the First Name column to string
cust_data['First Name'] = cust_data['First Name'].astype(str)

In [32]:
#Confirming datatype change in First Name column
cust_data['First Name'].apply(type).value_counts()

First Name
<class 'str'>    206209
Name: count, dtype: int64

In [33]:
#View missing values in a subset of the dataframe
df_nan_cust=cust_data[cust_data['First Name'].isnull()==True]

In [34]:
#Printing results of subset of NaN in the First Name column
df_nan_cust

,Customer_ID,First Name,Last Name,Gender,State,Age,Date Joined,Number of Dependents,Family Status,Income


Observation: Again, missing first names will most likely NOT impact the analysis; therefore, I will leave these NaN values as is.

In [36]:
#Looking for full duplicates
cust_dups=cust_data[cust_data.duplicated()]

In [37]:
print(cust_dups)

Empty DataFrame
Columns: [Customer_ID, First Name, Last Name, Gender, State, Age, Date Joined, Number of Dependents, Family Status, Income]
Index: []


Observation: No full duplicates!  Yay!

7. ***Combine customer data with ords_prods data***

Observation: Before writing the code to combine the dataframes, I looked at the three types of ways to combine the data: concatenating (combining dataframes that match in shape), joining (most useful when combining dataframes that have the same or related indicies), and merge (most useful for combining dataframes that do not match in shape). The dataframes that I will combine do not match in shape, so the best option is the merge method. I will use the INNER JOIN with the merge function because it will keep rows only where the key (Customer_ID) column is a match. It will exclude (drop) rows where a Key is not found for both. I thought this would be helpful, rather than the options for the other joins (Right, Left, Full Outer) because the other options would include NaN for the missing or non-matching value. Before I merge both dataframes on the "Customer_ID" column, I need to make sure that both dataframes have the same format for the header. That means I need to change the ords_prods_merge_grouped dataframe.

In [86]:
cust_data.shape

(206209, 10)

In [41]:
#Importing the ords_prods_merge_grouped dataframe
ords_prods_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merge_grouped.pkl'))

In [42]:
#Printing the header info
ords_prods_merge.head(0)

,order_id,customer_id,order_number,order_hour_of_day,days_since_prior_order,order_day_of_week_name,is_first_order,product_id,add_to_cart_order,reordered,...,price_range_loc,Busiest_Day,Busiest_Days,Busiest_Period_of_Day,max_order,loyalty_flag,avg_product_price,spend_flag,avg_days_between_orders,order_frequency_flag


In [43]:
#Renaming the column header from "customer_id" to "Customer_ID" in the ords_prods_merge_grouped dataframe so it matches the cust_data dataframe
ords_prods_merge = ords_prods_merge.rename(columns={ 
        'customer_id' : 'Customer_ID'})

In [44]:
#Confirming column has been renamed
ords_prods_merge.head(0)

,order_id,Customer_ID,order_number,order_hour_of_day,days_since_prior_order,order_day_of_week_name,is_first_order,product_id,add_to_cart_order,reordered,...,price_range_loc,Busiest_Day,Busiest_Days,Busiest_Period_of_Day,max_order,loyalty_flag,avg_product_price,spend_flag,avg_days_between_orders,order_frequency_flag


Observation: Success!  "Customer_ID" has been changed in the ords_prods_merge dataframe!  Now the common identifier column, "Customer_ID", matches
in both dataframes.

In [46]:
#Printing the column names for the ords_prods_merge dataframe (print() will list all column heads separated by comma for easy viewing)
print(ords_prods_merge.columns)

Index(['order_id', 'Customer_ID', 'order_number', 'order_hour_of_day',
       'days_since_prior_order', 'order_day_of_week_name', 'is_first_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'Busiest_Day', 'Busiest_Days',
       'Busiest_Period_of_Day', 'max_order', 'loyalty_flag',
       'avg_product_price', 'spend_flag', 'avg_days_between_orders',
       'order_frequency_flag'],
      dtype='object')


Observation:  I already included an indicator column ('_merge') when I joined the Orders dataframe to the Products dataframe.  I want to also include this indicator column when I join the ords_prods_merge dataframe to the cust_data dataframe.  To do that, I'll need to complete it four steps to stop getting the error messages for position arguments and incorrect use of indicator column.  First, rename existing _merge to _merge_old.  Next, merge ords_prods_merge and cust_data dataframes.  Then rename indicator column from _merge_old to _merge2.  Finally, use print.columns() to make sure merging (joining) of dataframes and renaming of the indicator column executed correctly.  

In [48]:
# Step 1: Rename the existing '_merge' column to '_merge_old'
ords_prods_merge = ords_prods_merge.rename(columns={'_merge': '_merge_old'})

In [49]:
# Step 2: Merge the dataframes with a new '_merge' column using indicator=True
ords_prods_cust_merge = ords_prods_merge.merge(cust_data, on='Customer_ID', indicator=True)

In [50]:
# Step 3: Rename the new '_merge' column to '_merge2'
ords_prods_cust_merge = ords_prods_cust_merge.rename(columns={'_merge': '_merge2'})

In [51]:
# Step 4: Verify the column names
print(ords_prods_cust_merge.columns)

Index(['order_id', 'Customer_ID', 'order_number', 'order_hour_of_day',
       'days_since_prior_order', 'order_day_of_week_name', 'is_first_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge_old',
       'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'Busiest_Day', 'Busiest_Days',
       'Busiest_Period_of_Day', 'max_order', 'loyalty_flag',
       'avg_product_price', 'spend_flag', 'avg_days_between_orders',
       'order_frequency_flag', 'First Name', 'Last Name', 'Gender', 'State',
       'Age', 'Date Joined', 'Number of Dependents', 'Family Status', 'Income',
       '_merge2'],
      dtype='object')


Observation:  It worked!  Correctly renamed the "_merge" indicator column in the ords_prods_merge dataframe to "_merge_old" and correctly
added an indicator column, "_merge2", in the cust_data dataframe.  Finally, joined the two dataframes correctly!!!!

8. ***Export new file (ords_prods_cust) as .pkl file to "Prepared Data"***

In [54]:
#Export as .pkl file to Prepared Data folder
ords_prods_cust_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_merge.pkl'))